# AI-CA1-Seaching

## Abstract: In this project, we are going to get familiar with searching problems. We try to model the problem in project description, such that it can be considered a searching problem. Then we use different searching algorithms for solving it and compare their execution time.

## Modeling problem:
## In following, search problem components for this problem are mentioned:
## State:
## Every state, consists of : position of unvisited mixtures(green sells), position of unvisited medicines(red cells), position of existing doctors(agents) and some othre information like cost of state or history of doctors path until this state. The first three information(position of unchecked green cells and red cells and position of existing doctors) are state specific and we use them to differentiate states from eachother in expanded states set(sometimes, dependent to algorithm we use, cost of state is also used for differentiation).
## Initial state:
## In this state, position of unvisited green and red cells, are identical to input green and red cells and we have only one existing agent at position of (0,0).( And if (0,0) is a red or green cell, this will be handled in preprocess of creating this state(in __init__ method) )
## Goal state:
## In this state, number of unchecked green cells of state is zero and all existing agents must be in up right corner(position (n-1,m-1)). The algorithm stops when this state is achieved.
## Action:
## Each action consists of moving one of agents of state that we are applying action on and checking if new position of moved agent is red or green to change unchecked red cells and add to existing agents or change unchecked green cells respectively(which leads to creation of new state). Note that in implementation, we apply acation, implicitly in preprocess of creation of new state(__init__ method).

## Implementation of State:
### \_\_init\_\_ : this is for creating new state: transmiting parent info or initiating initial state - applying action and check new agent position - assigning heuristic value for state if we use A*
### unique_str(_ids) : string id of state for differentiating with other states
### \_\_lt\_\_ : this method is needed in A* algorithm for priority queue and compares evaluation fuction value of 2 states
### is_goal_state : return true if number of unvisited green cells of state is zero and all agents are in target cell

In [1]:
# start position
START = (0,0)

class State:
    ALGORITHM = None
    ALPHA = 1
    def __init__(self, parent = None, direction = '', agent_index = 0, changed_pos = START):
        if parent != None:     # transmiting parent state info to new state
            self.history = parent.history
            self.cost = parent.cost
            self.agents = [*parent.agents]
            self.unchecked_greens = [*parent.unchecked_greens]
            self.unchecked_reds = [*parent.unchecked_reds]
            # applying action
            self.agents[agent_index] = changed_pos
            self.cost += 1
            self.history += str(agent_index) + direction + ' '

        else:                  # if it is initial state
            self.history = ''
            self.cost = 0
            self.agents = [START]
            self.unchecked_greens = [*green_cells]
            self.unchecked_reds = [*red_cells]

        # if new agent position is red cell
        if changed_pos in self.unchecked_reds:
            self.agents.append((n-1,0))
            self.unchecked_reds.remove(changed_pos)
            if (n-1,0) in self.unchecked_reds:
                self.agents.append((n-1,0))
                self.unchecked_reds.remove((n-1,0))
        # if new agent position is green cell
        elif changed_pos in self.unchecked_greens:
            self.unchecked_greens.remove(changed_pos)

        # heurisitc cost of state for when we use A* algorithm
        if State.ALGORITHM == A_STAR:
            self.heuristic = calculate_heuristic(self, State.ALPHA)

    def unique_str(self):      # string id of state
        return str([*self.agents, *self.unchecked_greens, *self.unchecked_reds])

    def unique_str_ids(self):      # string id of state for IDS algorithm
        return str([*self.agents, *self.unchecked_greens, *self.unchecked_reds, self.cost])
    
    # this method is needed in A* algorithm for priority queue
    # and compares evaluation fuction value of 2 states
    def __lt__(self, other):
        return self.heuristic + self.cost < other.heuristic + other.cost

    def is_goal_state(self):       # return if it is goal state
        return len(self.unchecked_greens) == 0 and len(set(self.agents)) == 1 and self.agents[0] == TARGET
    
    def print_state(self):      # printig state
        # print('position of doctors: ', list(enumerate(self.agents)))        # for debugging
        # print('position of unvisited mixtures: ', self.unchecked_greens)    # for debugging
        # print('position of unvisited medicines: ', self.unchecked_reds)     # for debugging
        print('agents path till this state: ', self.history)
        print('cost till this state: ', self.cost)
        print('-------------------------------------------------------\n')

## BFS Algorithm:
## In this algorithm, we use a queue(_states_queue_ in code) as a frontier and a set(_expanded_states_ in code) to avoid duplicating in expanding states that are already expanded( or pending to expand). First, we enqueue initial state to frontier. Then at each step of while loop, we dequeue a state from frontier and expand it. For each child state we first check whether it is goal state and if it is, we finish algorithm and return it as answer; But if it is not, after duplicate states checking, it may enqueue to frontier and therefore it _expanded_states_ set. Note that in _expanded_states_ set, what we're actually adding is string id of state, not state itself.

In [2]:
from queue import Queue

BFS = 0
def bfs(initial_state):
    expanded_states = set()
    states_queue = Queue()

    states_queue.put(initial_state)
    while True:
        curr_state = states_queue.get()
        for child_state in expand_state(curr_state):
            if child_state.is_goal_state():
                return child_state
            if child_state.unique_str() not in expanded_states:
                states_queue.put(child_state)
                expanded_states.add(child_state.unique_str())

## IDS Algorithm:
## In this Algorithm, we use DFS algorithm as subroutine, and at each step of while loop, we apply DFS on initial state with _max_level_ equal to max_cost(starting from 0 and increasing it at each step) until we returned goal state is not None. Finally we return final state.

In [3]:
IDS = 1
def ids(initial_state):
    max_cost = 0
    goal_state = None
    while goal_state == None:
        expanded_states = set()
        expanded_states.add(initial_state)
        goal_state = dfs(initial_state, max_cost, expanded_states)
        max_cost+=1
    return goal_state

## Description of _dfs_ function:
## This fucntion iterates over child states of its argument state; For each child state, first it checks if child state is goal state(and if it is, fucntion returns it), then if it is not a duplicate state, adds its id to _expanded_states_ and recursively apply _dfs_ on it, only if the cost of child state is less than max_level(to satisfy IDS algorithm limit); If called _dfs_ returns valid answer, it returns that answer otherwise it moves to next child state.
### Note that in _dfs_, we use a _expanded_states_ set too, with exact same purpose in BFS algoritm but with little difference in string id of state; In this one, cost of state is also part of id of state.

In [4]:
def dfs(state, max_level, expanded_states):
    to_return = None
    for child_state in expand_state(state):
            if child_state.is_goal_state():
                return child_state
            if child_state.unique_str_ids() not in expanded_states:
                expanded_states.add(child_state.unique_str_ids())
                if child_state.cost >= max_level:
                    continue
                to_return = dfs(child_state, max_level, expanded_states)
                if to_return != None:
                    return to_return
    return to_return

## A* Algorithm:
## This algorithm is very similar to BFS algorithm; The only change is the type of queue that is used for forntier. In this one, we use priority queue or min_heap as frontier. For inserting in this heap, states will be compare to eachother based on evaluation function value(current cost + heuristic value) that is implemented in State class(_\_\_lt\_\__ method). For popping from heap, the state with lowest evaluation function value, will pop up and will be expanded.

In [5]:
import heapq
from bisect import insort

A_STAR = 2
def A_star(initial_state):
    expanded_states = set()
    heap = []

    insort(heap, initial_state)
    while True:
        curr_state = heapq.heappop(heap)
        for child_state in expand_state(curr_state):
            if child_state.is_goal_state():
                return child_state
            if child_state.unique_str() not in expanded_states:
                insort(heap, child_state)
                expanded_states.add(child_state.unique_str())

## Heuristic funciton:
$\begin{equation}
 heuristic=\sum_{agent}^\ min(Manhattan(agent,target), min(Manhattan(agent, mixtures)))
 \end{equation}$
 
## As equation shows, heuristic of each state is calaulted by summation of this value for all agents of state:
## minimum of Manhattan distance of agtent and each item in set of $[taregt, mixture_1, mixture_2, ... , mixure_c]$
## It is consistent: Real cost of moving form parent state to child state is 1(just one moved agent in child state). Also $heuristic(parent) - heuristic(child) \leq 1$ ; Because in child state, just one of agents has moved and only manhattan distance of that agent and nearest position of above set, has decreased by 1(actually $heuristic(parent) - heuristic(child)$ can sometimes be negative. Because if moved agent in child state is reached its nearest position, now the manhattan related to that agent may increase.)

In [6]:
def calculate_heuristic(state, alpha = 1):
    heurisitc = 0
    for agent_pos in state.agents:
        agent_heurisitc = (n+m-(2+agent_pos[0]+agent_pos[1]))
        for green_pos in state.unchecked_greens:
            agent_heurisitc = min(agent_heurisitc, abs(agent_pos[0]-green_pos[0])+abs(agent_pos[1]-green_pos[1]))
        heurisitc += agent_heurisitc
    return heurisitc * alpha

## _expand_state_ fuction:
## This fucntion generates all possible child states that its argument state can have(the process of applying action will be in state object instantiating). A child state of a given state can achieved by simply moving one of existing agents of state to valid neighbor position; _get_neighbor_position_ fucntion return such position and it will be _None_ if it is invalid(like out of bound or a bloked cell). Note that for each agent position in outer loop, we have a little optimization in such way that if all green cells are visited(which means all agents must go to target position) and agent is already in taregt positon, we do not move that agent.

In [7]:
def expand_state(state):
    global COUNT
    for index, agent_pos in enumerate(state.agents):
        if agent_pos == TARGET and len(state.unchecked_greens) == 0:
            continue 
        for direction in ['U','R','D','L']:
            new_pos = get_neighbor_pos(direction, agent_pos)
            if new_pos != None:
                COUNT += 1
                yield State(state, direction, index, new_pos)

def get_neighbor_pos(direction, pos):
    x = pos[0]
    y = pos[1]
    if direction == 'U':
        x+=1
    elif direction == 'R':
        y+=1
    elif direction == 'D':
        x-=1
    else:
        y-=1
    if 0 <= x and x < n and 0 <= y and y < m and (x,y) not in blocks:
            return (x,y)
    return None

## Runnig algorithms

In [8]:
from time import time
COUNT = 0

## Test 1

## Reading input

In [9]:
n, m, c, k, d, TARGET = 0,0,0,0,0,0
green_cells = []
red_cells = []
blocks = []
with open('test1.in', 'r') as fd:
    n, m = map(int, fd.readline().strip().split())
    TARGET = (n-1,m-1)
    c, k = map(int, fd.readline().strip().split())
    for _ in range(c):
        i, j = map(int, fd.readline().strip().split())
        green_cells.append((i,j))
    for _ in range(k):
        i, j = map(int, fd.readline().strip().split())
        red_cells.append((i,j))
    d = int(fd.readline().strip())
    for _ in range(d):
        i, j = map(int, fd.readline().strip().split())
        blocks.append((i,j))

### BFS

In [10]:
State.ALGORITHM = BFS
initial_state = State()

exe_time = time()
answer = bfs(initial_state)
exe_time = time() - exe_time
print('bfs-test1 :')
print('execution time: %s seconds' % exe_time)
answer.print_state()


average_bfs_time = 0
COUNT = 0
# run1
exe_time = time()
bfs_answer = bfs(initial_state)
exe_time = time() - exe_time
average_bfs_time += exe_time
state_count_bfs = COUNT
# run2
exe_time = time()
bfs(initial_state)
exe_time = time() - exe_time
average_bfs_time += exe_time
# run3
exe_time = time()
bfs(initial_state)
exe_time = time() - exe_time
average_bfs_time += exe_time

average_bfs_time *= 1000
average_bfs_time /= 3

bfs-test1 :
execution time: 0.02414393424987793 seconds
agents path till this state:  0R 0R 0R 0U 0U 0U 1D 1U 1R 1R 1R 
cost till this state:  11
-------------------------------------------------------



### IDS

In [11]:
State.ALGORITHM = IDS

exe_time = time()
answer = ids(initial_state)
exe_time = time() - exe_time
print('ids-test1 :')
print('execution time: %s seconds' % exe_time)
answer.print_state()


average_ids_time = 0
COUNT = 0
# run1
exe_time = time()
ids_answer = ids(initial_state)
exe_time = time() - exe_time
average_ids_time += exe_time
state_count_ids = COUNT
# run2
exe_time = time()
ids(initial_state)
exe_time = time() - exe_time
average_ids_time += exe_time
# run3
exe_time = time()
ids(initial_state)
exe_time = time() - exe_time
average_ids_time += exe_time

average_ids_time *= 1000
average_ids_time /= 3

ids-test1 :
execution time: 0.15030574798583984 seconds
agents path till this state:  0R 0R 0R 0U 0U 0U 1D 1U 1R 1R 1R 
cost till this state:  11
-------------------------------------------------------



### A*

In [12]:
State.ALGORITHM = A_STAR

exe_time = time()
answer = A_star(initial_state)
exe_time = time() - exe_time
print('A*-test1 :')
print('execution time: %s seconds' % exe_time)
answer.print_state()


average_Astar_time = 0
COUNT = 0
# run1
exe_time = time()
Astar_answer = A_star(initial_state)
exe_time = time() - exe_time
average_Astar_time += exe_time
state_count_Astar = COUNT
# run2
exe_time = time()
A_star(initial_state)
exe_time = time() - exe_time
average_Astar_time += exe_time
# run3
exe_time = time()
A_star(initial_state)
exe_time = time() - exe_time
average_Astar_time += exe_time

average_Astar_time *= 1000
average_Astar_time /= 3

A*-test1 :
execution time: 0.012711048126220703 seconds
agents path till this state:  0R 1D 1R 1R 1R 0R 0R 1U 0U 0U 0U 
cost till this state:  11
-------------------------------------------------------



### A* alpha = 2, 3

In [13]:
# alpha = 2
State.ALPHA = 2
exe_time = time()
alpha2_answer = A_star(initial_state)
exe_time = time() - exe_time
print('A* alpha=2-test1 :')
print('execution time: %s seconds' % exe_time)
alpha2_answer.print_state()

average_alpha2_time = 0
COUNT = 0
# run1
exe_time = time()
A_star(initial_state)
exe_time = time() - exe_time
average_alpha2_time += exe_time
state_count_alpha2 = COUNT
# run2
exe_time = time()
A_star(initial_state)
exe_time = time() - exe_time
average_alpha2_time += exe_time
# run3
exe_time = time()
A_star(initial_state)
exe_time = time() - exe_time
average_alpha2_time += exe_time

average_alpha2_time *= 1000
average_alpha2_time /= 3


# alpha = 3
State.ALPHA = 3
exe_time = time()
alpha3_answer = A_star(initial_state)
exe_time = time() - exe_time
print('A* alpha=3-test1 :')
print('execution time: %s seconds' % exe_time)
alpha3_answer.print_state()

average_alpha3_time = 0
COUNT = 0
# run1
exe_time = time()
A_star(initial_state)
exe_time = time() - exe_time
average_alpha3_time += exe_time
state_count_alpha3 = COUNT
# run2
exe_time = time()
A_star(initial_state)
exe_time = time() - exe_time
average_alpha3_time += exe_time
# run3
exe_time = time()
A_star(initial_state)
exe_time = time() - exe_time
average_alpha3_time += exe_time

average_alpha3_time *= 1000
average_alpha3_time /= 3

A* alpha=2-test1 :
execution time: 0.03864741325378418 seconds
agents path till this state:  0R 0R 0L 0U 0L 1R 1R 1R 0U 0U 0R 0R 0R 
cost till this state:  13
-------------------------------------------------------

A* alpha=3-test1 :
execution time: 0.00991511344909668 seconds
agents path till this state:  0R 1D 1R 1U 1R 1R 0R 0R 0U 0U 0U 
cost till this state:  11
-------------------------------------------------------



### table info

In [14]:
print('BFS - Test1 :')
print('answer distance(cost) :', bfs_answer.cost)
print('#visited_states :', state_count_bfs)
print('average exe time(ms) :', average_bfs_time)
print('-------------------------------------------------')
print('IDS - Test1 :')
print('answer distance(cost) :', ids_answer.cost)
print('#visited_states :', state_count_ids)
print('average exe time(ms) :', average_ids_time)
print('--------------------------------------------------')
print('A* - Test1 :')
print('answer distance(cost) :', Astar_answer.cost)
print('#visited_states :', state_count_Astar)
print('average exe time(ms) :', average_Astar_time)
print('--------------------------------------------------')
print('Alpha2 - Test1 :')
print('answer distance(cost) :', alpha2_answer.cost)
print('#visited_states :', state_count_alpha2)
print('average exe time(ms) :', average_alpha2_time)
print('--------------------------------------------------')
print('Alpha3 - Test1 :')
print('answer distance(cost) :', alpha3_answer.cost)
print('#visited_states :', state_count_alpha3)
print('average exe time(ms) :', average_alpha3_time)

BFS - Test1 :
answer distance(cost) : 11
#visited_states : 3788
average exe time(ms) : 17.742156982421875
-------------------------------------------------
IDS - Test1 :
answer distance(cost) : 11
#visited_states : 23377
average exe time(ms) : 103.68410746256511
--------------------------------------------------
A* - Test1 :
answer distance(cost) : 11
#visited_states : 1173
average exe time(ms) : 10.8186403910319
--------------------------------------------------
Alpha2 - Test1 :
answer distance(cost) : 13
#visited_states : 1775
average exe time(ms) : 13.06454340616862
--------------------------------------------------
Alpha3 - Test1 :
answer distance(cost) : 11
#visited_states : 1585
average exe time(ms) : 10.529836018880209


## Test2

## Reading input

In [15]:
n, m, c, k, d, TARGET = 0,0,0,0,0,0
green_cells = []
red_cells = []
blocks = []
with open('test2.in', 'r') as fd:
    n, m = map(int, fd.readline().strip().split())
    TARGET = (n-1,m-1)
    c, k = map(int, fd.readline().strip().split())
    for _ in range(c):
        i, j = map(int, fd.readline().strip().split())
        green_cells.append((i,j))
    for _ in range(k):
        i, j = map(int, fd.readline().strip().split())
        red_cells.append((i,j))
    d = int(fd.readline().strip())
    for _ in range(d):
        i, j = map(int, fd.readline().strip().split())
        blocks.append((i,j))

### BFS

In [16]:
State.ALGORITHM = BFS
initial_state = State()

exe_time = time()
answer = bfs(initial_state)
exe_time = time() - exe_time
print('bfs-test2 :')
print('execution time: %s seconds' % exe_time)
answer.print_state()


average_bfs_time = 0
COUNT = 0
# run1
exe_time = time()
bfs_answer = bfs(initial_state)
exe_time = time() - exe_time
average_bfs_time += exe_time
state_count_bfs = COUNT
# run2
exe_time = time()
bfs(initial_state)
exe_time = time() - exe_time
average_bfs_time += exe_time
# run3
exe_time = time()
bfs(initial_state)
exe_time = time() - exe_time
average_bfs_time += exe_time

average_bfs_time *= 1000
average_bfs_time /= 3

bfs-test2 :
execution time: 2.9450762271881104 seconds
agents path till this state:  0R 0R 0U 0U 0U 0R 1D 1R 1R 1U 1R 
cost till this state:  11
-------------------------------------------------------



### IDS

In [17]:
State.ALGORITHM = IDS

exe_time = time()
answer = ids(initial_state)
exe_time = time() - exe_time
print('ids-test2 :')
print('execution time: %s seconds' % exe_time)
answer.print_state()


average_ids_time = 0
COUNT = 0
# run1
exe_time = time()
ids_answer = ids(initial_state)
exe_time = time() - exe_time
average_ids_time += exe_time
state_count_ids = COUNT
# run2
exe_time = time()
ids(initial_state)
exe_time = time() - exe_time
average_ids_time += exe_time
# run3
exe_time = time()
ids(initial_state)
exe_time = time() - exe_time
average_ids_time += exe_time

average_ids_time *= 1000
average_ids_time /= 3

ids-test2 :
execution time: 6.005188226699829 seconds
agents path till this state:  0R 0R 0U 0U 0U 0R 1D 1R 1R 1U 1R 
cost till this state:  11
-------------------------------------------------------



### A*

In [18]:
State.ALGORITHM = A_STAR
State.ALPHA = 1
exe_time = time()
answer = A_star(initial_state)
exe_time = time() - exe_time
print('A*-test2 :')
print('execution time: %s seconds' % exe_time)
answer.print_state()


average_Astar_time = 0
COUNT = 0
# run1
exe_time = time()
Astar_answer = A_star(initial_state)
exe_time = time() - exe_time
average_Astar_time += exe_time
state_count_Astar = COUNT
# run2
exe_time = time()
A_star(initial_state)
exe_time = time() - exe_time
average_Astar_time += exe_time
# run3
exe_time = time()
A_star(initial_state)
exe_time = time() - exe_time
average_Astar_time += exe_time

average_Astar_time *= 1000
average_Astar_time /= 3

A*-test2 :
execution time: 0.1378180980682373 seconds
agents path till this state:  0R 0R 0U 0U 0R 0U 1D 1R 1R 1U 1R 
cost till this state:  11
-------------------------------------------------------



### A* alpha=2,3

In [19]:
# alpha = 2
State.ALPHA = 2
exe_time = time()
alpha2_answer = A_star(initial_state)
exe_time = time() - exe_time
print('A* alpha=2-test2 :')
print('execution time: %s seconds' % exe_time)
alpha2_answer.print_state()

average_alpha2_time = 0
COUNT = 0
# run1
exe_time = time()
A_star(initial_state)
exe_time = time() - exe_time
average_alpha2_time += exe_time
state_count_alpha2 = COUNT
# run2
exe_time = time()
A_star(initial_state)
exe_time = time() - exe_time
average_alpha2_time += exe_time
# run3
exe_time = time()
A_star(initial_state)
exe_time = time() - exe_time
average_alpha2_time += exe_time

average_alpha2_time *= 1000
average_alpha2_time /= 3


# alpha = 3
State.ALPHA = 3
exe_time = time()
alpha3_answer = A_star(initial_state)
exe_time = time() - exe_time
print('A* alpha=3-test2 :')
print('execution time: %s seconds' % exe_time)
alpha3_answer.print_state()

average_alpha3_time = 0
COUNT = 0
# run1
exe_time = time()
A_star(initial_state)
exe_time = time() - exe_time
average_alpha3_time += exe_time
state_count_alpha3 = COUNT
# run2
exe_time = time()
A_star(initial_state)
exe_time = time() - exe_time
average_alpha3_time += exe_time
# run3
exe_time = time()
A_star(initial_state)
exe_time = time() - exe_time
average_alpha3_time += exe_time

average_alpha3_time *= 1000
average_alpha3_time /= 3

A* alpha=2-test2 :
execution time: 0.05873394012451172 seconds
agents path till this state:  0R 0R 0U 0U 0U 0R 1D 1R 1R 1U 1R 
cost till this state:  11
-------------------------------------------------------

A* alpha=3-test2 :
execution time: 0.07074403762817383 seconds
agents path till this state:  0R 0R 1D 1R 1R 1U 1R 0U 0U 0U 0R 
cost till this state:  11
-------------------------------------------------------



### table info

In [20]:
print('BFS - Test2 :')
print('answer distance(cost) :', bfs_answer.cost)
print('#visited_states :', state_count_bfs)
print('average exe time(ms) :', average_bfs_time)
print('-------------------------------------------------')
print('IDS - Test2 :')
print('answer distance(cost) :', ids_answer.cost)
print('#visited_states :', state_count_ids)
print('average exe time(ms) :', average_ids_time)
print('--------------------------------------------------')
print('A* - Test2 :')
print('answer distance(cost) :', Astar_answer.cost)
print('#visited_states :', state_count_Astar)
print('average exe time(ms) :', average_Astar_time)
print('--------------------------------------------------')
print('Alpha2 - Test2 :')
print('answer distance(cost) :', alpha2_answer.cost)
print('#visited_states :', state_count_alpha2)
print('average exe time(ms) :', average_alpha2_time)
print('--------------------------------------------------')
print('Alpha3 - Test2 :')
print('answer distance(cost) :', alpha3_answer.cost)
print('#visited_states :', state_count_alpha3)
print('average exe time(ms) :', average_alpha3_time)

BFS - Test2 :
answer distance(cost) : 11
#visited_states : 482477
average exe time(ms) : 2749.4908968607583
-------------------------------------------------
IDS - Test2 :
answer distance(cost) : 11
#visited_states : 1258418
average exe time(ms) : 5874.538421630859
--------------------------------------------------
A* - Test2 :
answer distance(cost) : 11
#visited_states : 10171
average exe time(ms) : 117.23923683166504
--------------------------------------------------
Alpha2 - Test2 :
answer distance(cost) : 11
#visited_states : 4244
average exe time(ms) : 51.93789800008138
--------------------------------------------------
Alpha3 - Test2 :
answer distance(cost) : 11
#visited_states : 6077
average exe time(ms) : 94.07083193461101


## Test3

## Reading input

In [21]:
n, m, c, k, d, TARGET = 0,0,0,0,0,0
green_cells = []
red_cells = []
blocks = []
with open('test3.in', 'r') as fd:
    n, m = map(int, fd.readline().strip().split())
    TARGET = (n-1,m-1)
    c, k = map(int, fd.readline().strip().split())
    for _ in range(c):
        i, j = map(int, fd.readline().strip().split())
        green_cells.append((i,j))
    for _ in range(k):
        i, j = map(int, fd.readline().strip().split())
        red_cells.append((i,j))
    d = int(fd.readline().strip())
    for _ in range(d):
        i, j = map(int, fd.readline().strip().split())
        blocks.append((i,j))

### BFS

In [22]:
State.ALGORITHM = BFS
initial_state = State()

exe_time = time()
answer = bfs(initial_state)
exe_time = time() - exe_time
print('bfs-test3 :')
print('execution time: %s seconds' % exe_time)
answer.print_state()


average_bfs_time = 0
COUNT = 0
# run1
exe_time = time()
bfs_answer = bfs(initial_state)
exe_time = time() - exe_time
average_bfs_time += exe_time
state_count_bfs = COUNT
# run2
exe_time = time()
bfs(initial_state)
exe_time = time() - exe_time
average_bfs_time += exe_time
# run3
exe_time = time()
bfs(initial_state)
exe_time = time() - exe_time
average_bfs_time += exe_time

average_bfs_time *= 1000
average_bfs_time /= 3

bfs-test3 :
execution time: 3.6254703998565674 seconds
agents path till this state:  0U 0U 0R 0R 0D 0R 0R 0R 0U 0U 0U 0U 1R 1R 1R 1D 1U 1R 1R 
cost till this state:  19
-------------------------------------------------------



### IDS

In [23]:
State.ALGORITHM = IDS

exe_time = time()
answer = ids(initial_state)
exe_time = time() - exe_time
print('ids-test3 :')
print('execution time: %s seconds' % exe_time)
answer.print_state()


average_ids_time = 0
COUNT = 0
# run1
exe_time = time()
ids_answer = ids(initial_state)
exe_time = time() - exe_time
average_ids_time += exe_time
state_count_ids = COUNT
# run2
exe_time = time()
ids(initial_state)
exe_time = time() - exe_time
average_ids_time += exe_time
# run3
exe_time = time()
ids(initial_state)
exe_time = time() - exe_time
average_ids_time += exe_time

average_ids_time *= 1000
average_ids_time /= 3

ids-test3 :
execution time: 12.5356285572052 seconds
agents path till this state:  0U 0U 0R 0R 0D 0R 0R 0R 0U 0U 0U 0U 1R 1R 1R 1D 1U 1R 1R 
cost till this state:  19
-------------------------------------------------------



### A*

In [24]:
State.ALGORITHM = A_STAR
State.ALPHA = 1
exe_time = time()
answer = A_star(initial_state)
exe_time = time() - exe_time
print('A*-test3 :')
print('execution time: %s seconds' % exe_time)
answer.print_state()


average_Astar_time = 0
COUNT = 0
# run1
exe_time = time()
Astar_answer = A_star(initial_state)
exe_time = time() - exe_time
average_Astar_time += exe_time
state_count_Astar = COUNT
# run2
exe_time = time()
A_star(initial_state)
exe_time = time() - exe_time
average_Astar_time += exe_time
# run3
exe_time = time()
A_star(initial_state)
exe_time = time() - exe_time
average_Astar_time += exe_time

average_Astar_time *= 1000
average_Astar_time /= 3

A*-test3 :
execution time: 0.9421095848083496 seconds
agents path till this state:  0U 0U 0R 0R 1R 1R 1R 0D 0R 1D 1U 0R 1R 1R 0R 0U 0U 0U 0U 
cost till this state:  19
-------------------------------------------------------



### A* alpha=2,3

In [25]:
# alpha = 2
State.ALPHA = 2
exe_time = time()
alpha2_answer = A_star(initial_state)
exe_time = time() - exe_time
print('A* alpha=2-test3 :')
print('execution time: %s seconds' % exe_time)
alpha2_answer.print_state()

average_alpha2_time = 0
COUNT = 0
# run1
exe_time = time()
A_star(initial_state)
exe_time = time() - exe_time
average_alpha2_time += exe_time
state_count_alpha2 = COUNT
# run2
exe_time = time()
A_star(initial_state)
exe_time = time() - exe_time
average_alpha2_time += exe_time
# run3
exe_time = time()
A_star(initial_state)
exe_time = time() - exe_time
average_alpha2_time += exe_time

average_alpha2_time *= 1000
average_alpha2_time /= 3


# alpha = 3
State.ALPHA = 3
exe_time = time()
alpha3_answer = A_star(initial_state)
exe_time = time() - exe_time
print('A* alpha=3-test3 :')
print('execution time: %s seconds' % exe_time)
alpha3_answer.print_state()

average_alpha3_time = 0
COUNT = 0
# run1
exe_time = time()
A_star(initial_state)
exe_time = time() - exe_time
average_alpha3_time += exe_time
state_count_alpha3 = COUNT
# run2
exe_time = time()
A_star(initial_state)
exe_time = time() - exe_time
average_alpha3_time += exe_time
# run3
exe_time = time()
A_star(initial_state)
exe_time = time() - exe_time
average_alpha3_time += exe_time

average_alpha3_time *= 1000
average_alpha3_time /= 3

A* alpha=2-test3 :
execution time: 0.34029412269592285 seconds
agents path till this state:  0U 0U 0R 0R 0D 0R 0R 1R 1R 1R 0R 0U 0U 0U 0U 1D 1R 1R 1U 
cost till this state:  19
-------------------------------------------------------

A* alpha=3-test3 :
execution time: 0.22321605682373047 seconds
agents path till this state:  0U 0U 0R 0R 0D 0R 0R 1R 1R 1R 0R 0U 1D 1U 1R 0U 0U 1R 0U 
cost till this state:  19
-------------------------------------------------------



### table info

In [26]:
print('BFS - Test3 :')
print('answer distance(cost) :', bfs_answer.cost)
print('#visited_states :', state_count_bfs)
print('average exe time(ms) :', average_bfs_time)
print('-------------------------------------------------')
print('IDS - Test3 :')
print('answer distance(cost) :', ids_answer.cost)
print('#visited_states :', state_count_ids)
print('average exe time(ms) :', average_ids_time)
print('--------------------------------------------------')
print('A* - Test3 :')
print('answer distance(cost) :', Astar_answer.cost)
print('#visited_states :', state_count_Astar)
print('average exe time(ms) :', average_Astar_time)
print('--------------------------------------------------')
print('Alpha2 - Test3 :')
print('answer distance(cost) :', alpha2_answer.cost)
print('#visited_states :', state_count_alpha2)
print('average exe time(ms) :', average_alpha2_time)
print('--------------------------------------------------')
print('Alpha3 - Test3 :')
print('answer distance(cost) :', alpha3_answer.cost)
print('#visited_states :', state_count_alpha3)
print('average exe time(ms) :', average_alpha3_time)

BFS - Test3 :
answer distance(cost) : 19
#visited_states : 591821
average exe time(ms) : 3519.8185443878174
-------------------------------------------------
IDS - Test3 :
answer distance(cost) : 19
#visited_states : 2499345
average exe time(ms) : 11962.80344327291
--------------------------------------------------
A* - Test3 :
answer distance(cost) : 19
#visited_states : 73921
average exe time(ms) : 1007.0658524831136
--------------------------------------------------
Alpha2 - Test3 :
answer distance(cost) : 19
#visited_states : 25032
average exe time(ms) : 312.46693929036456
--------------------------------------------------
Alpha3 - Test3 :
answer distance(cost) : 19
#visited_states : 13961
average exe time(ms) : 165.57788848876953


## Observation
## According to table info for each test, we can say that first 3 algorithms are able to find optimal solution, but last two ones (A* with alpha) do not necessarily find(as we can see cost in A* - alpha=2 for Test1). In execution time, we can say IDS > BFS > A* . Time execution of A* with alpha is less than A* . The implementation of A* is very similar to BFS; It only use different type of queue and has heuristic function. Implementation of IDS is seems to be more difficalt than other ones. But if we have memeroy limits, it's better to use this one over bfs. In overall A* seems to be best choice; Because it is informed search algorithm and implementation is not that muth hard. The key point in this algorithm is hueristic function that if it is consistent, then we get best result. If we have some time limits in solving problem we can use this algorithm with weight(alpha) 
